<a href="https://colab.research.google.com/github/budubezimeni/SkillFactory/blob/main/SF_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re

In [183]:
from sklearn.metrics import mean_absolute_error

# Нужные функции
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

# Проверка на число
def check_numeric(x):
    try:
        float(x)
        return True
    except:
        return False

In [ ]:
RANDOM_SEED = 42

In [ ]:
!pip install -U pandas-profiling

In [ ]:
import pandas_profiling

In [ ]:
df = pd.read_csv('/content/data.csv', engine='python', encoding='utf-8', error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   status        337267 non-null  object
 1   private pool  4181 non-null    object
 2   propertyType  342452 non-null  object
 3   street        377183 non-null  object
 4   baths         270847 non-null  object
 5   homeFacts     377185 non-null  object
 6   fireplace     103115 non-null  object
 7   city          377151 non-null  object
 8   schools       377185 non-null  object
 9   sqft          336608 non-null  object
 10  zipcode       377185 non-null  object
 11  beds          285903 non-null  object
 12  state         377185 non-null  object
 13  stories       226470 non-null  object
 14  mls-id        24942 non-null   object
 15  PrivatePool   40311 non-null   object
 16  MlsId         310305 non-null  object
 17  target        374704 non-null  object
dtypes: object(18)
memory usa

In [ ]:
profile = pandas_profiling.ProfileReport(df)


In [ ]:
profile

# **Начинаем обработку столбцов**

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
# Разделяем столбцы на числовые, бинарные, категориальные
bin_cols = []
cat_cols = []
num_cols = []

Удалим все строки с отсутствуюшим **target**

In [ ]:
df.drop(df[df.target.isnull()].index, inplace=True)

In [62]:
df.target = df.target.replace('[^\d.]','',regex=True)
df.target = df.target.apply(lambda x: float(x) if check_numeric(x) else 0)

**status**
Столбец имеет пропуски и большое количество уникальных значений

In [ ]:
# Приводим к нижнему регистру
df.status = df.status.str.lower()

In [ ]:
# Заменяем очевидные значения
status_dict = {'p': 'pending', ' / auction': 'auction', 'a active': 'active', 'under contract show': 'under contract   showing'}
df.status = df.status.replace(status_dict)

In [ ]:
status_array = df.status.value_counts()

In [ ]:
# Заменяем редкие значения на other
for status_value in status_array.items():
  if status_value[1] < 100:
    df.status = df.status.replace({status_value[0]: 'other'})

In [ ]:
# Заполняем пропуски
df.status = df.status.fillna('other')

In [ ]:
cat_cols.append('status')

Объединяем столбцы **private pool** и **PrivatePool** в столбец **PrivatePool_combined**

In [ ]:
df['PrivatePool_combined'] = df.apply (lambda row: (row['PrivatePool']=='Yes') | (row['PrivatePool']=='yes') | (row['private pool']=='Yes'), axis=1)

In [ ]:
df = df.drop(['PrivatePool', 'private pool'], 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
bin_cols.append('PrivatePool_combined')

**city** столбец имеет пропуски

In [ ]:
# Заполняем пропущенные города по индексу
for row in df[df.city.isnull()].zipcode.items():
  df.loc[df.index == row[0], 'city'] = df[df.zipcode==row[1]].city.value_counts().reset_index()['index'][0]

**propertyType** имеет пропуски

In [ ]:
# Приводим к нижнему регистру
df.propertyType = df.propertyType.str.lower()

In [ ]:
# Заполняем пропуски
df.propertyType = df.propertyType.fillna('other')

In [ ]:
# Нормализуем популярные значения
df.loc[df['propertyType'].str.contains(r'^(?=.*single)(?=.*family)'), 'propertyType'] = 'single family'
df.loc[df['propertyType'].str.contains(r'^(?=.*multi)(?=.*family)'), 'propertyType'] = 'multi family'
df.loc[df['propertyType'].str.contains(r'^(?=.*condo)'), 'propertyType'] = 'condo'
df.loc[df['propertyType'].str.contains(r'^(?=.*mobile)'), 'propertyType'] = 'mobile'
df.loc[df['propertyType'].str.contains('1 story|one story'), 'propertyType'] = 'one story'
df.loc[df['propertyType'].str.contains('2 stor|two stor'), 'propertyType'] = 'two story'

In [ ]:
# Заменяем редкие значения на other
propertyType_array = df.propertyType.value_counts()
for propertyType_value in propertyType_array.items():
  if propertyType_value[1] < 100:
    df.propertyType = df.propertyType.replace({propertyType_value[0]: 'other'})

In [ ]:
cat_cols.append('propertyType')

**street** имеет пропущенные значения и много уникальных

In [ ]:
# Приводим к нижнему регистру
df.street = df.street.str.lower()

In [ ]:
# Заполняем пропуски
df.street = df.street.fillna('unknown')

In [ ]:
df.loc[df['street'].str.contains(' st| street'), 'street'] = 'street'
df.loc[df['street'].str.contains(' dr| drive'), 'street'] = 'drive'
df.loc[df['street'].str.contains(' ave| avenue'), 'street'] = 'avenue'
df.loc[df['street'].str.contains(' blvd| boulevard'), 'street'] = 'boulevard'
df.loc[df['street'].str.contains(' pl| place'), 'street'] = 'place'
df.loc[df['street'].str.contains(' rd| road'), 'street'] = 'road'
df.loc[df['street'].str.contains(' ln| lane'), 'street'] = 'lane'
df.loc[df['street'].str.contains(' cir| circus'), 'street'] = 'circus'
df.loc[df['street'].str.contains(' ct| court'), 'street'] = 'court'
df.loc[df['street'].str.contains(' hwy| highway'), 'street'] = 'highway'
df.loc[df['street'].str.contains(' way'), 'street'] = 'way'
df.loc[df['street'].str.contains(' sq| square'), 'street'] = 'square'
df.loc[df['street'].str.contains(' end'), 'street'] = 'end'
df.loc[df['street'].str.contains(' trl| trail'), 'street'] = 'trail'
df.loc[df['street'].str.contains(' ter'), 'street'] = 'territory'
df.loc[df['street'].str.contains(' pkwy'), 'street'] = 'parkway'
df.loc[df['street'].str.contains(' lane'), 'street'] = 'lane'
df.loc[df['street'].str.contains(' pass'), 'street'] = 'pass'

In [ ]:
# Заменяем редкие значения на other
street_array = df.street.value_counts()
for street_value in street_array.items():
  if street_value[1] < 1000:
    df.street = df.street.replace({street_value[0]: 'other'})

In [ ]:
cat_cols.append('street')

**baths** имеет пропуски и смешанные значения

In [ ]:
# Выделим цифровое значение из строки
df.baths = df.baths.replace('\+','',regex=True)
df.baths = df.baths.replace('[^\d.-]','',regex=True)
df.baths = df.baths.apply(lambda x: float(x) if check_numeric(x) else 0)

In [ ]:
# Нормализуем выбивающиеся значения
df.baths = df.baths.apply(lambda x: x/1000 if x>100 else x)

In [ ]:
# Заполним пропуски медианой
df.baths = df.baths.fillna(df.baths.median())

In [ ]:
num_cols.append('baths')

**fireplace** имеет пропуски и неоднородные значения. Имеет смысл разбить на наличие/отсутствие

In [ ]:
df.fireplace = df.fireplace.apply(lambda x: False if pd.isnull(x) else True)

In [ ]:
bin_cols.append('fireplace')

**sqft** выделим числовое занчение и заполним пропуски 0

In [ ]:
df.sqft = df.sqft.replace('[^\d.]','',regex=True)

In [ ]:
df.sqft = df.sqft.apply(lambda x: float(x) if check_numeric(x) else 0)

In [ ]:
df.sqft = df.sqft.fillna(0)

In [ ]:
num_cols.append('sqft')

**zipcode** индекс, не имеет пропусков

In [ ]:
cat_cols.append('zipcode')

**homeFacts** не имеет  пропусков, и имеет одинаковое кол-во фактов, так что разобьем на отдельные признаки

In [ ]:
def jsonHomeFacts(strFacts, idx):
  match = re.findall(r"{.+}", strFacts)
  if match:
    dict1 = eval(match[0]).get('atAGlanceFacts')
    json1 = json.dumps(dict1)
    res1 = json.loads(json1)
    dictVal = pd.json_normalize(res1)
    return dictVal['factValue'][idx]

In [ ]:
# На основе этих данных можно будет сделать дополнительный признак Отопления или валидацию площади дома
# for i in range(0,6):
#   df['homeFacts_' + str(i)] = df.homeFacts.apply(lambda x: jsonHomeFacts(x, i))

**schools** json данные без пропусков. Выделим новые признаки

In [ ]:
# Проверка на целое число
def check_int(x):
    try:
        int(x)
        return True
    except:
        return False

In [ ]:
def schoolsMaxRating(x):
  jsonSchools = json.dumps(eval(x))
  dataSchools = json.loads(jsonSchools)
  dictVal = pd.json_normalize(dataSchools)
  max_rating = 0
  for rating in dictVal['rating'].array[0]:
    if rating.find('/') == 1:
      rating = rating.split('/')[0]
    if check_int(rating):
      if int(rating) > max_rating:
        max_rating = int(rating)
    
  return max_rating

In [ ]:
df['schools_max_rating'] = df.schools.apply(lambda x: schoolsMaxRating(x))

In [ ]:
num_cols.append('schools_max_rating')

In [ ]:
def schoolsCount(x):
  jsonSchools = json.dumps(eval(x))
  dataSchools = json.loads(jsonSchools)
  dictVal = pd.json_normalize(dataSchools)
  return len(dictVal['rating'].array[0])

In [ ]:
df['schools_cnt'] = df.schools.apply(lambda x: schoolsCount(x))

In [ ]:
df['schools_cnt'] = df['schools_cnt'].apply(lambda x: x if x<10 else 10)

In [ ]:
num_cols.append('schools_cnt')

In [ ]:
def schoolsMinDistance(x):
  jsonSchools = json.dumps(eval(x))
  dataSchools = json.loads(jsonSchools)
  dictVal = pd.json_normalize(dataSchools)
  min_distance = 0
  for distance in dictVal['data.Distance'].array[0]:
    distance = re.sub(r"[^\d.]", "", distance)
    if check_numeric(distance):
      if (float(distance) < min_distance) | (min_distance == 0):
        min_distance = float(distance)
  return min_distance

In [ ]:
df['schools_min_distance'] = df.schools.apply(lambda x: schoolsMinDistance(x))

In [ ]:
df['schools_min_distance'] = df['schools_min_distance'].apply(lambda x: round(x,0))

In [ ]:
df['schools_min_distance'] = df['schools_min_distance'].apply(lambda x: 10 if x>9 else x)

In [ ]:
num_cols.append('schools_min_distance')

**city** и **state** будут коррелировать с zipcode, но иметь меньшую информативность в виду точности позиционирования.

**stories** имеет много пропусков и частично имеется в **propertyType**

**MlsId и mls_id** код агента, много уникальных значений, не несущих информации.

**Удалим их**

In [ ]:
df.drop(['city', 'state', 'stories', 'MlsId', 'mls-id'], axis=1, inplace=True)

In [ ]:
df.drop(['homeFacts', 'schools'], axis=1, inplace=True)

**beds** кол-во спален, выделим число из наиболее популярных вариантов, остальные заменим медианой

In [ ]:
# Заменяем редкие значения на 0
beds_array = df.beds.value_counts()
for beds_value in beds_array.items():
  if beds_value[1] < 1000:
    df.beds = df.beds.replace({beds_value[0]: '0.0'})

In [ ]:
def bedsValue(x):
  bedsCount = re.sub(r"[^\d.]", "", x)
  if check_numeric(bedsCount):
    return float(bedsCount)
  return 0.0

In [ ]:
df.beds = df.beds.fillna('0.0')

In [ ]:
df['beds'] = df.beds.apply(lambda x: bedsValue(x))

In [ ]:
df.beds = df.beds.replace({0.0: df['beds'].median()})

In [ ]:
num_cols.append('beds')

Добавим признак цены за кв фут **price_per_sqft**

In [ ]:
df['price_per_sqft'] = df['target']/df.sqft

In [ ]:
df['price_per_sqft'] = df['price_per_sqft'].replace([np.inf, -np.inf], 0)

In [ ]:
num_cols.append('price_per_sqft')

In [ ]:
df['price_per_sqft'].quantile(.9)

483.8095622713206

Заменим значение индекса на классификацию индекса по средней цене недвижимости **zipcode**

In [76]:
zipcode_avg_value = df.loc[(df.price_per_sqft>0), 'price_per_sqft'].quantile(0.5)
zipcode_expensive_value = df.loc[(df.price_per_sqft>0), 'price_per_sqft'].quantile(0.9)

In [84]:
def getZipcodeCluster(x):
  if df.loc[(df.zipcode == x)&(df.price_per_sqft>0), 'price_per_sqft'].mean() < zipcode_avg_value:
    return 'cheap'
  elif df.loc[(df.zipcode == x)&(df.price_per_sqft>0), 'price_per_sqft'].mean() > zipcode_expensive_value:
    return 'very_expensive'
  else:
    return 'expensive'

In [88]:
zipcode_array = df.zipcode.value_counts()
for zipcode_value in zipcode_array.items():
  df.zipcode = df.zipcode.replace({zipcode_value[0]: getZipcodeCluster(zipcode_value[0])})

In [93]:
cat_cols.append('zipcode')

In [73]:
# Загружаем из файла чтобы не проводить предобработку
# df = pd.read_csv('/content/data_cleaned2.csv', engine='python', encoding='utf-8', error_bad_lines=False)
# df.columns.values[0] = 'id'
# df.drop(['id'], axis=1, inplace=True) 
# cat_cols = ['status', 'propertyType', 'street', 'zipcode']
# num_cols = ['baths',
#  'sqft',
#  'schools_max_rating',
#  'schools_cnt',
#  'schools_min_distance',
#  'beds',
#  'price_per_sqft']
# bin_cols = ['PrivatePool_combined', 'fireplace']

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# **CatboostRegressor**

In [ ]:
train, validate, test = np.split(df, [int(.7*len(df)), int(.85*len(df))])

In [ ]:
train.loc[train.baths>10, 'baths'] = 10
max_sqft = train.sqft.quantile(0.999)
train.loc[train.sqft>max_sqft, 'sqft'] = max_sqft

In [ ]:
y_train = train.target
train.drop(['target'], axis=1, inplace=True)
X_train = train
y_test = test.target
test.drop(['target'], axis=1, inplace=True)
X_test = test
y_val = validate.target
validate.drop(['target'], axis=1, inplace=True)
X_val = validate

In [ ]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 1.2 MB/s 


In [ ]:
from catboost import CatBoostRegressor

In [114]:
model = CatBoostRegressor(iterations=500,
                          learning_rate=0.05,
                          depth=5,
                          eval_metric='MAPE',
                          random_seed = RANDOM_SEED,
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500)

# В Catboost передаем категориальные переменные без обработки
cat_features = cat_cols
# Fit model
model.fit(X_train, np.log(y_train), cat_features, eval_set=(X_test, np.log(y_test)), use_best_model=True)
# Get predictions
preds = model.predict(X_test)

0:	learn: 0.0680601	test: 0.0682004	best: 0.0682004 (0)	total: 235ms	remaining: 1m 57s
1:	learn: 0.0657916	test: 0.0659362	best: 0.0659362 (1)	total: 421ms	remaining: 1m 44s
2:	learn: 0.0637440	test: 0.0638871	best: 0.0638871 (2)	total: 611ms	remaining: 1m 41s
3:	learn: 0.0616846	test: 0.0618257	best: 0.0618257 (3)	total: 831ms	remaining: 1m 43s
4:	learn: 0.0597680	test: 0.0599099	best: 0.0599099 (4)	total: 996ms	remaining: 1m 38s
5:	learn: 0.0579262	test: 0.0580659	best: 0.0580659 (5)	total: 1.17s	remaining: 1m 36s
6:	learn: 0.0562046	test: 0.0563505	best: 0.0563505 (6)	total: 1.35s	remaining: 1m 35s
7:	learn: 0.0546848	test: 0.0548357	best: 0.0548357 (7)	total: 1.55s	remaining: 1m 35s
8:	learn: 0.0531344	test: 0.0532858	best: 0.0532858 (8)	total: 1.74s	remaining: 1m 34s
9:	learn: 0.0516073	test: 0.0517483	best: 0.0517483 (9)	total: 1.93s	remaining: 1m 34s
10:	learn: 0.0501106	test: 0.0502605	best: 0.0502605 (10)	total: 2.11s	remaining: 1m 33s
11:	learn: 0.0486982	test: 0.0488479	best

In [115]:
print(f"TEST mape: {(mape(y_test, np.exp(preds)))}%")

TEST mape: 1.7967872287796862%


**TEST mape: 1.7967872287796862%**

In [116]:
preds_val = model.predict(X_val)

In [118]:
print(f"Validation mape: {(mape(y_val, np.exp(preds_val)))}%")

Validation mape: 7.165714989962479%


**Validation mape: 7.165714989962479%**

# **LinearRegression (Baseline)**

In [155]:
from sklearn import linear_model

In [156]:
model = linear_model.LinearRegression()

In [174]:
model.fit(X_train, y_train)

LinearRegression()

In [175]:
preds = model.predict(X_test)

In [173]:
np.exp(preds).value_counts()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: RuntimeWarning: overflow encountered in exp
  """Entry point for launching an IPython kernel.


AttributeError: ignored

In [176]:
print(f"TEST mape: {(mape(y_test, preds))}%")

TEST mape: 21.494000327445242%


**TEST mape: 21.494000327445242%**

In [177]:
print(f"TEST mae: {(mean_absolute_error(y_test, preds))}")

TEST mae: 624289.200913713


**TEST mae: 624289.200913713**

In [179]:
preds_val = model.predict(X_val)

In [180]:
print(f"Validation mape: {(mape(y_val, preds_val))}%")

Validation mape: 33.044020153163785%


**Validation mape: 33.044020153163785%**

In [182]:
print(f"Validation mae: {(mean_absolute_error(y_val, preds_val))}")

Validation mae: 543588.6908563941


**Validation mae: 543588.6908563941**

# **XGBoost**

In [ ]:
df_xg = df.copy()

# Label Encoding
for column in cat_cols:
    df_xg[column] = df_xg[column].astype('category').cat.codes

for column in bin_cols:
    df_xg[column] = df_xg[column].astype('category').cat.codes
    
# One-Hot Encoding: в pandas есть готовая функция - get_dummies.
df_xg = pd.get_dummies(df_xg, columns=cat_cols, dummy_na=False)
df_xg = pd.get_dummies(df_xg, columns=bin_cols, dummy_na=False)

In [ ]:
train, validate, test = np.split(df_xg, [int(.7*len(df_xg)), int(.85*len(df_xg))])

In [ ]:
train.loc[train.baths>10, 'baths'] = 10
max_sqft = train.sqft.quantile(0.999)
train.loc[train.sqft>max_sqft, 'sqft'] = max_sqft

In [ ]:
y_train = train.target
train.drop(['target'], axis=1, inplace=True)
X_train = train
y_test = test.target
test.drop(['target'], axis=1, inplace=True)
X_test = test
y_val = validate.target
validate.drop(['target'], axis=1, inplace=True)
X_val = validate

In [123]:
import xgboost as xgb

In [152]:
model = xgb.XGBRegressor(verbosity=0)

In [153]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [154]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
# report result
print('Mean Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  UserWarning,


KeyboardInterrupt: ignored

In [145]:
model.fit(X_train, np.log(y_train))

XGBRegressor(verbosity=0)

In [146]:
preds = model.predict(X_test)

In [147]:
print(f"TEST mape: {(mape(y_test, np.exp(preds)))}%")

TEST mape: 1.7308398834813148%


In [148]:
print(f"TEST mae: {(mean_absolute_error(y_test, np.exp(preds)))}")

TEST mae: 120880.79397099829


**TEST mape: 1.7308398834813148%**

In [149]:
preds_val = model.predict(X_val)

In [150]:
print(f"Validation mape: {(mape(y_val, np.exp(preds_val)))}%")

Validation mape: 7.246956482973879%


In [151]:
print(f"Validation mae: {(mean_absolute_error(y_test, np.exp(preds_val)))}")

Validation mae: 726245.1694075448


**Validation mape: 7.246956482973879%**